In [1]:
import torch
import torch.nn as nn 
import numpy as np 
import os 
import sys 
from torch.autograd import Variable 


from jigsaw_data import dataset

In [2]:
from glob import glob
from time import time

from nltk.tokenize import RegexpTokenizer
from collections import defaultdict

import torch
import torch.nn as nn   
import torch.utils.data as data
from torch.autograd import Variable
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F

import tqdm
import os
import sys
import numpy as np
import pandas as pd
from PIL import Image
import numpy.random as random

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

import pandas as pd  

import torchtext 
import spacy
from bs4 import BeautifulSoup
import dill 

from torchtext import vocab
from torchtext.data import Example
from torchtext.datasets import language_modeling
from torchtext.data import Field, BucketIterator, TabularDataset, ReversibleField, Dataset
from torchtext.vocab import GloVe
from sklearn.model_selection import train_test_split
from itertools import chain

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

from SimpleLSTM import SimpleLSTMBaseline, SimpleBiLSTMBaseline
from jigsaw_data import dataset
from EncRNN import EncoderRNN2, EncoderRNN
from attn import Attn, Attn2

from attn_models import Attention, Classifier

batch_size = 16


In [32]:
class EncoderRNN(nn.Module):

    def __init__(self, emb_dim, h_dim, v_size, gpu=True, v_vec=None, batch_first=True):
        super(EncoderRNN, self).__init__()
        self.gpu = gpu
        self.h_dim = h_dim
        self.embed = nn.Embedding(v_size, emb_dim)
        if v_vec is not None:
            self.embed.weight.data.copy_(v_vec)

        self.lstm = nn.LSTM(emb_dim, h_dim, batch_first=batch_first,
                            bidirectional=True)

    def init_hidden(self, b_size):
        h0 = Variable(torch.zeros(1*2, b_size, self.h_dim))
        c0 = Variable(torch.zeros(1*2, b_size, self.h_dim))
        if self.gpu:
            h0 = h0.cuda()
            c0 = c0.cuda()
        return (h0, c0)

    def forward(self, sentence, lengths=None):
        self.hidden = self.init_hidden(sentence.size(0))
        emb = self.embed(sentence)
        packed_emb = emb

        #print("packed_emb shape", packed_emb.shape)

        if lengths is not None:
            lengths = lengths.view(-1).tolist()
            packed_emb = nn.utils.rnn.pack_padded_sequence(emb, lengths)

        out, hidden = self.lstm(packed_emb, self.hidden)
        print("hidden shape", len(hidden))
        print(hidden[0].shape)
        print(hidden[1].shape)
        #rint("hidden shape", hidden.shape)


        if lengths is not None:
            out = nn.utils.rnn.pad_packed_sequence(out)[0]

        out = out[:, :, :self.h_dim] + out[:, :, self.h_dim:]
        #print("final out shape", out.shape)

        return out


In [5]:
train, val, TEXT, LABEL = dataset()

train_iter, val_iter = BucketIterator.splits(
    (train, val), # we pass in the datasets we want the iterator to draw data from
    batch_sizes=(batch_size,batch_size),
    device=torch.device("cuda"), # if you want to use the GPU, specify the GPU number here
    sort_key=lambda x: len(x.comment_text), # the BucketIterator needs to be told what function it should use to group the data.
    sort_within_batch=False,
    repeat=False # we pass repeat=False because we want to wrap this Iterator layer.
)

train_dl = BatchWrapper(train_iter, "comment_text", ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"])
valid_dl = BatchWrapper(val_iter, "comment_text", ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"])

x,y = next( iter( train_dl ) )
print("sentence shape", x.shape)
#print(y.shape)

pandas df size (159571, 8)
                 id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   

   severe_toxic  obscene  threat  insult  identity_hate  
0             0        0       0       0              0  
1             0        0       0       0              0  
- - - - - - - - - - - - - - - - - - - - 
train val length 7000 3000
time to build vocab 4.214108943939209
length of vocaburary 17045 torch.Size([17045, 100])
- - - - - - - - - - - - - - - - - - - - 
* most common words.
[('article', 3328), ('page', 2838), ('wikipedia', 2096), ('talk', 1939), ('would', 1870), ('like', 1833), ('one', 1767), ('please', 1676), ('dont', 1598), ('see', 1363), ('also', 1272), ('im', 1259), ('know', 1217), ('think', 1212), ('edit', 1116), ('people', 1085), ('use', 1047), ('articles', 992), ('fuck', 964), ('time', 946)]
sent

In [33]:
enc = EncoderRNN(  emb_dim = 100 , h_dim = 100, v_size = len(TEXT.vocab), v_vec=TEXT.vocab.vectors  )

In [34]:
enc.cuda()

EncoderRNN(
  (embed): Embedding(17045, 100)
  (lstm): LSTM(100, 100, batch_first=True, bidirectional=True)
)

In [35]:
out = enc(x)

hidden shape 2
torch.Size([2, 221, 100])
torch.Size([2, 221, 100])


In [9]:
out.shape

torch.Size([221, 16, 100])

In [15]:
x.shape

torch.Size([221, 16])

In [18]:

class EncoderRNN2(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1, dropout=0.1, v_vec=None, bidirectional=True):
        super(EncoderRNN2, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.bidirectional = bidirectional
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        if v_vec is not None:
            self.embedding.weight.data.copy_(v_vec)

        if n_layers > 1:
            self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=self.dropout, bidirectional=bidirectional)
        else:
            self.gru = nn.GRU(hidden_size, hidden_size, n_layers, bidirectional=bidirectional)

    def forward(self, input_seqs, input_lengths=None, hidden=None):
        # Note: we run this all at once (over multiple batches of multiple sequences)
        embedded = self.embedding(input_seqs)
        packed = embedded

        if input_lengths is not None:
            packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        
        outputs, hidden = self.gru(packed, hidden)

        print("gru hidden shape to be sum for bidirectional", hidden.shape)

        if input_lengths is not None:
            outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs) # unpack (back to padded)
        
        #outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:] # Sum bidirectional outputs
        return outputs, hidden


In [22]:
enc2 = EncoderRNN2(input_size=len(TEXT.vocab), hidden_size=100, v_vec=TEXT.vocab.vectors, n_layers=2) 
enc2.cuda()

EncoderRNN2(
  (embedding): Embedding(17045, 100)
  (gru): GRU(100, 100, num_layers=2, dropout=0.1, bidirectional=True)
)

In [65]:
out2, hidden = enc2(x)
print(out2.shape)

gru hidden shape to be sum for bidirectional torch.Size([4, 16, 100])
torch.Size([221, 16, 200])


In [41]:
hidden[-1]

tensor([[ 0.0426, -0.2212, -0.0368,  ...,  0.1450, -0.0869, -0.0506],
        [ 0.1403,  0.0165, -0.0034,  ...,  0.0803, -0.0486, -0.0476],
        [ 0.0810, -0.0106, -0.0277,  ...,  0.0551,  0.0474, -0.2034],
        ...,
        [ 0.0672, -0.1555, -0.0425,  ..., -0.0685, -0.0493, -0.0780],
        [-0.1141, -0.2359, -0.0423,  ...,  0.0423,  0.1731, -0.0449],
        [ 0.1971,  0.0121,  0.0700,  ...,  0.0938,  0.0787, -0.0480]],
       device='cuda:0', grad_fn=<SelectBackward>)

In [42]:
hidden[-2]

tensor([[-0.0177,  0.1250, -0.1365,  ..., -0.1202, -0.0509,  0.0653],
        [-0.0360,  0.0929, -0.1022,  ..., -0.0933, -0.0279,  0.0671],
        [-0.0220,  0.1139, -0.1297,  ..., -0.1178, -0.0641,  0.0630],
        ...,
        [-0.0228,  0.1143, -0.1380,  ..., -0.1235, -0.0496,  0.0723],
        [-0.0164,  0.1205, -0.1194,  ..., -0.1197, -0.0558,  0.0801],
        [-0.0306,  0.1071, -0.1201,  ..., -0.1086, -0.0530,  0.0628]],
       device='cuda:0', grad_fn=<SelectBackward>)

In [69]:
h = torch.cat( [ hidden[-1], hidden[-2] ], dim=1)
print(h.shape)

torch.Size([16, 200])


In [66]:
class Attention(nn.Module):
    def __init__(self, query_dim, key_dim, value_dim):
        super(Attention, self).__init__()
        self.scale = 1. / np.sqrt(query_dim)

    def forward(self, query, keys, values):
        # Query = [BxQ]
        # Keys = [TxBxK]
        # Values = [TxBxV]
        # Outputs = a:[TxB], lin_comb:[BxV]

        # Here we assume q_dim == k_dim (dot product attention)

        query = query.unsqueeze(1) # [BxQ] -> [Bx1xQ]
        print(query.shape)
        
        keys = keys.transpose(0,1).transpose(1,2) # [TxBxK] -> [BxKxT]
        print(keys.shape)
        
        energy = torch.bmm(query, keys) # [Bx1xQ]x[BxKxT] -> [Bx1xT]
        print(energy.shape)
        
        energy = F.softmax(energy.mul_(self.scale), dim=2) # scale, normalize
        print(energy.shape)
        
        values = values.transpose(0,1) # [TxBxV] -> [BxTxV]
        print(values.shape)
        
        linear_combination = torch.bmm(energy, values).squeeze(1) #[Bx1xT]x[BxTxV] -> [BxV]
        print(linear_combination.shape)
        
        return energy, linear_combination


In [67]:
attention = Attention(200,200,200)

In [68]:
e, linear_combination = attention(h,out2,out2)

torch.Size([16, 1, 200])
torch.Size([16, 200, 221])
torch.Size([16, 1, 221])
torch.Size([16, 1, 221])
torch.Size([16, 221, 200])
torch.Size([16, 200])


In [63]:
e.shape

torch.Size([16, 1, 221])

In [64]:
linear_combination.shape

torch.Size([16, 200])

In [70]:
nn.Linear(200,10)(linear_combination)

RuntimeError: Expected object of backend CPU but got backend CUDA for argument #4 'mat1'